In [1]:
import numpy as np
from numpy import genfromtxt

import csv
import os

import deepSI
from deepSI import System_data

import pysindy
from pysindy import SINDy

from torch import nn
import torch

import matplotlib.pyplot as plt

from scipy.io import loadmat

from deepSI.utils import simple_res_net, feed_forward_nn

from utils import SINDy_model

c:\Users\20173928\Miniconda3\envs\sindy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
